In [163]:
import TeledyneLeCroyPy
from matplotlib import pyplot as plt
import requests
import numpy as np
import time
import pandas as pd
import h5py
import pickle
import json
import os
import pandas as pd
import gspread
from google.oauth2 import service_account
import json
import subprocess

credentials = service_account.Credentials.from_service_account_file(
    '../.google_credentials.json')
scope = ['https://spreadsheets.google.com/feeds']
creds_scope = credentials.with_scopes(scope)
client = gspread.authorize(creds_scope)

sheet = client.open_by_key('1idBnsYG4pHdHZ2kDbq-S3PBUAf0ewFEzKXOtCTGDxyU')
log = sheet.worksheet("log")

#
# data saved in logbook
# https://docs.google.com/spreadsheets/d/1idBnsYG4pHdHZ2kDbq-S3PBUAf0ewFEzKXOtCTGDxyU
#


start = time.time()
while True:
    try:
        #o = TeledyneLeCroyPy.LeCroyWaveRunner('TCPIP0::192.168.99.103::inst0::INSTR')
        o = TeledyneLeCroyPy.LeCroyWaveRunner('TCPIP0::192.168.189.115::inst0::INSTR')
        break
    except:
        end = time.time()
        print("waiting for connection... "+str(int(end-start))+"s", end="\r")
        time.sleep(0.1)
        pass
print("Connected", o.idn) # Prings e.g. LECROY,WAVERUNNER9254M,LCRY4751N40408,9.2.0

Connected LECROY,WR8108HD,LCRY5004N60822,10.5.0


In [164]:
def append_record_to_hdf5(filename, record_id, record_data):
    with h5py.File(filename, 'a') as hdf_file:
        group = hdf_file.create_group(str(record_id))
        for key, value in record_data.items():
            group.create_dataset(key, data=value)
            
def append_record_to_hdf5_(filename, record_id, record_data):
#
# deve essere un array, ma non funziona quindi i record vanno messi fra []. c'e' qulcosa che non ho cpaito 
# esempio: dict2save['H'+str(channel)]=[json.dumps(data['wavedesc'], default=str)]
#
    with h5py.File(filename, 'a') as hdf_file:
        group = hdf_file.create_group(str(record_id))
        for key, value in record_data.items():
            group.create_dataset(key, data=value,
                      compression='gzip',
                      compression_opts=9)
            
def append_record_to_pickle(filename, record):
    # Read existing data from the pickle file
    try:
        with open(filename, 'rb') as file:
            existing_data = pickle.load(file)
    except FileNotFoundError:
        existing_data = []
    # Append the new record to the existing data
    existing_data.append(record)

    # Write the updated data back to the pickle file
    with open(filename, 'wb') as file:
        pickle.dump(existing_data, file)
            
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
from pymemcache.client.base import Client


def mc_get_str(ip, key):

    client = Client((ip, 11211))
    result = client.get(key)
    return result.decode("utf-8")

In [166]:
EVENTS= 1000
DSHOW = False
FTYPE = 'H5'
CHANNELS = 4
PATH = '/Volumes/WC/data/'

user_input = input("Run description ")
if user_input:
    start_desc=user_input
else:
    start_desc=''

user_input = input("Number of channels: [{}] ".format(CHANNELS)).lower()
if user_input:
    channels=int(user_input)
else:
    channels=CHANNELS
user_input = input("Number of events: [{}] ".format(EVENTS)).lower()
if user_input:
    events=int(user_input)
else:
    events=EVENTS

user_input = input("file type: [{}] else PKL".format(FTYPE)).lower()
if user_input=='pkl':
    pkl = True
else:
    pkl = False

#pkl=True

# reading logbook
logdf=pd.DataFrame.from_dict(log.get_all_records())
last_run = logdf.run.values[-1]
run = last_run+1
    
    
if pkl:
    filename = "run_{:05d}.pkl".format(run)
else:
    filename = "run_{:05d}.h5".format(run)
filepath = PATH+filename
    
#########################
start = time.time()
dt = 0

if os.path.exists(filepath):
    os.remove(filepath)
    print('File removed:', filepath)
else:
    print('Writing on new file:', filepath)

beam = mc_get_str(ip='192.168.198.164', key='BTFDATA_PADME')
# updating startup condition
start_date = time.strftime("%Y-%m-%dT%H:%M:%S", time.gmtime())
start_epoch = time.time()
logdf = logdf._append({'run': run, 'start_desc': start_desc, 'start_date': start_date, 'start_epoch': start_epoch, 
                       'filename': filename, 'beam': beam}, ignore_index=True)


for event in range(events):
    dict2save = {}
    triggers = []
    try:
        elapsed = time.time()-start
        if event >0:
            trate = elapsed/event
        else:
            trate = 0 
        print('Triggers acquired: {:d}, elapsed {:.1f} s, Tr Hz: {:.1f}, storing time: {:.2f} s'.format(event,elapsed, trate, dt), end="\r")
        o.wait_for_single_trigger() # Halt the execution until there is a trigger.
        dt1 = time.time()
        for channel in range(1, channels+1):
            data = o.get_waveform(n_channel=channel)
            if pkl:
                triggers.append(data)
            else:
                dict2save['H'+str(channel)]=json.dumps(data['wavedesc'], default=str)
                dict2save['T'+str(channel)]=json.dumps(data['trigtime'])
                dict2save['W'+str(channel)]=json.dumps(data['waveforms'], cls=NumpyEncoder)
            if DSHOW:
                # show first waveform per 
                x = data['waveforms'][0]['Time (s)']/1e-9
                y = data['waveforms'][0]['Amplitude (V)']/1e-3
                plt.plot(x,y, label="C"+str(n_channel))

        
        if pkl:
            append_record_to_pickle(filepath, triggers)
        else:
            dict2save['epoch']=data['wavedesc']['TRIGGER_TIME'].timestamp()
            append_record_to_hdf5(filepath, event, dict2save)
        if DSHOW:
            t = data['wavedesc']['TRIGGER_TIME'].strftime('%m/%d/%Y %H:%M:%S')
            plt.legend()
            plt.title(t)
            plt.xlabel('ns')
            plt.ylabel('mV')
            plt.show()
        dt =  time.time() - dt1
    except KeyboardInterrupt:
        print ("\nstopping DAQ...")
        break

print ("\nUpating metadata run info...")
# updating ending condition
end_date = time.strftime("%Y-%m-%dT%H:%M:%S", time.gmtime())
logdf.at[run, 'end_date']=end_date
end_epoch = time.time()
logdf.at[run, 'end_epoch']=end_epoch
logdf.at[run, 'events']=event+1

print ("\nUploding file on cloud in background...")
subprocess.Popen(['./uploadFile.py', filepath], stdout=None, stderr=None, stdin=None)

user_input = input("closing remarks? (if any [])")
if user_input:
    end_desc=user_input
else:
    end_desc=''
logdf.at[run, 'end_desc']=end_desc

# updating logbook
print ("\nUpdating logbook...")
log.update([logdf.columns.values.tolist()] + logdf.values.tolist())

print("\nRUN {:} ACQUIRED".format(filename))
print("\nDAQ STOP")

Run description 
Number of channels: [4] 
Number of events: [1000] 10
file type: [H5] else PKL
File removed: /Volumes/WC/data/run_00004.h5
Triggers acquired: 9, elapsed 6.1 s, Tr Hz: 0.7, storing time: 0.49 s
Upating metadata run info...

Uploding file on cloud in background...
closing remarks? (if any [])

Updating logbook...

RUN run_00004.h5 ACQUIRED

DAQ STOP


In [158]:
from pymemcache.client.base import Client


def mc_get_str(ip, key):

    client = Client((ip, 11211))
    result = client.get(key)
    return result.decode("utf-8")

beam = mc_get_str(ip='192.168.198.164', key='BTFDATA_PADME')

In [161]:
json.dumps(beam)

'"BTFDATA_PADME,3796045233.093758,0,1,0,1,0,0,449.21,0.00,0.00,0,1,1,1,298.00,286.00,0.00\\r\\nBTFKEY_PADME,KEY_TIM,KEY_STA,DHPTB101_OK,BTF_PHA,DHSTB001_OK,DHSTB002_OK,DHRTB102_OK,DHSTB001_ENE,DHSTB002_ENE,DHRTB102_CUR,LINAC_PHA,MOD_OK,GUN_OK,BTFDAQ_OK,DHRTB101_CUR,DHSTB001_CUR,DHSTB002_CUR\\r\\n\\r\\n"'

In [149]:
import h5py

# Aprire il file HDF5
with h5py.File(filepath, 'r') as hdf_file:
    # Ottenere il dataset di interesse
    dataset = hdf_file["0"]
    
    # Verificare le proprietà del dataset
    print("Compression:", dataset.compression)
    print("Compression options:", dataset.compression_opts)

AttributeError: 'Group' object has no attribute 'compression'

In [ ]:
dataset

In [125]:
for key, value in dict2save.items():
    print(key, type([value]))

In [103]:
dict2save['H1']

'{"DESCRIPTOR_NAME": "WAVEDESC", "TEMPLATE_NAME": "LECROY_2_3", "COMM_TYPE": 1, "COMM_ORDER": 0, "WAVE_DESCRIPTOR": 346, "USER_TEXT": 0, "RES_DESC1": 0, "TRIGTIME_ARRAY": 0, "RIS_TIME_ARRAY": 0, "RES_ARRAY1": 0, "WAVE_ARRAY_1": 20004, "WAVE_ARRAY_2": 0, "RES_ARRAY2": 0, "RES_ARRAY3": 0, "INSTRUMENT_NAME": "LECROYWaveRunner", "INSTRUMENT_NUMBER": 20543, "TRACE_LABEL": "", "RESERVED1": 0, "RESERVED2": 10002, "WAVE_ARRAY_COUNT": 10002, "PNTS_PER_SCREEN": 10000, "FIRST_VALID_PNT": 0, "LAST_VALID_PNT": 10001, "FIRST_POINT": 0, "SPARSING_FACTOR": 1, "SEGMENT_INDEX": 0, "SUBARRAY_COUNT": 1, "SWEEPS_PER_ACQ": 1, "POINTS_PER_PAIR": 0, "PAIR_OFFSET": 0, "VERTICAL_GAIN": 1.2972400327271316e-06, "VERTICAL_OFFSET": 0.03420000150799751, "MAX_VALUE": 30579.0, "MIN_VALUE": -30835.0, "NOMINAL_BITS": 8, "NOM_SUBARRAY_COUNT": 1, "HORIZ_INTERVAL": 5.00000006675716e-11, "HORIZ_OFFSET": -3.1404710717026614e-07, "PIXEL_OFFSET": -3.14e-07, "VERTUNIT": "V", "HORIZ_UNCERTAINTY": 9.999999960041972e-13, "TRIGGER_

In [116]:
f.close()

In [118]:
f=h5py.File('prova.h5', 'w')
f.create_dataset("zipped", dict2save['H1'], dtype='string', compression="gzip")
f.close()

OSError: Unable to create file (unable to truncate a file which is already open)

In [ ]:
binary_data = pickle.dumps(data)
record_data = {'c1': binary_data}
len(binary_data)

In [79]:
#append_record_to_hdf5('/Volumes/WC/data/test2.h5', 1, binary_data)

with h5py.File('/Volumes/WC/data/test2.h5', 'a') as hdf_file:
    group = hdf_file.create_group(str(7))
    for key, value in record_data.items():
        group.create_dataset(key, data=binary_data, dtype='byte')

ValueError: invalid literal for int() with base 10: b'\x80\x04\x95(\x05\x00\x00\x00\x00\x00\x00}\x94(\x8c\x08wavedesc\x94}\x94(\x8c\x0fDESCRIPTOR_NAME\x94\x8c\x08WAVEDESC\x94\x8c\rTEMPLATE_NAME\x94\x8c\nLECROY_2_3\x94\x8c\tCOMM_TYPE\x94K\x01\x8c\nCOMM_

In [111]:
a =(100, 100)

In [114]:
type(a)

tuple

In [132]:
!pip install -U h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: h5py
    Found existing installation: h5py 3.10.0
    Uninstalling h5py-3.10.0:
      Successfully uninstalled h5py-3.10.0


In [136]:
import gzip
import shutil
with open(filepath, 'rb') as f_in:
    with gzip.open(filepath+'.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [143]:
import io
bio = io.BytesIO()
with h5py.File(bio, 'w') as f:
    f['dataset'] = data


TypeError: Object dtype dtype('O') has no native HDF5 equivalent